In [1]:
import requests
from bs4 import BeautifulSoup
from trafilatura import fetch_url, extract

In [86]:
url="http://www.snuh.org/health/nMedInfo/nList.do?pageIndex=1"

response  = requests.get(url)

In [87]:
response.status_code

200

In [88]:
response.text

' \r\n\r\n\r\n\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n\t\r\n\t\t\r\n\t\r\n<head>\r\n\t<meta charset="UTF-8">\r\n\t<title>서울대학교병원</title>\r\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n\t<meta name="format-detection" content="telephone=no">\r\n\t\r\n\t\r\n\t\r\n\t<meta name="keywords" content="Seoul National University Hospital, korea, child, kid, cancer, 서울대학교병원, 서울대병원, 어린이병원, 암병원" />\t\r\n\t<link rel="shortcut icon" href="/asset/img/common/snuh.ico" type="image/x-icon"/>\r\n\r\n\t<!--[if lt IE 9]>\r\n\t\t<script src="/asset/js/html5shiv.min.js"></script>\r\n\t\t<link rel="stylesheet" href="/asset/css/notosanskr-eot.min.css">\r\n\t<![endif]-->\r\n\t<script src="/asset/js/webfont.min.js"></script>\r\n\t<link rel="stylesheet" href="/asset/css/base.min.css">\r\n\t<link rel="stylesheet" href="/asset/css/common.css?ver=220705"><!-- 220705 비압축파일로 변경 -->\r\n\t<link rel="stylesheet" href="/asset/css/content.min.css">\r\n \t<link rel="stylesheet" href="/asset/css/blog-renewal.

In [89]:
soup = BeautifulSoup(response.text, features="lxml")

In [ ]:
soup

In [91]:
#http://www.snuh.org/health/nMedInfo/nView.do?medid=AA000491

ROOT_URL="http://www.snuh.org/health/nMedInfo"

titles= soup.find_all("div",{"class": "title"})

for title in titles:
    sub_url = title.find("a")["href"]
    url= f"{ROOT_URL}/{sub_url}"

print(url)


http://www.snuh.org/health/nMedInfo/./nView.do?medid=BA000074


In [92]:
#http://www.snuh.org/health/nMedInfo/nView.do?medid=AA000491
#get url and sub url in div title
ROOT_URL="http://www.snuh.org/health/nMedInfo"

titles= soup.find_all("div",{"class": "title"})

sub_url = titles[0].find("a")["href"]
url= f"{ROOT_URL}/{sub_url}"

response = requests.get(url) #must re-request coz now we are using the url root/sub not the first one
soup = BeautifulSoup(response.text, features="lxml")

print(url)

http://www.snuh.org/health/nMedInfo/./nView.do?medid=AA000491


In [ ]:
soup

In [94]:
#get titles

title = soup.find("div", {"class": "viewTitle"}).text
title = " ".join(title.split())
print(title)

ABO 신생아용혈성질환 [ABO hemolytic disease of the newborn]


In [95]:
medi_info = soup.find("div",{"class": "mediInfoView"})

In [ ]:
medi_info

In [100]:
#get desc n medic dept

meta_info={} #save in dict type

for info in medi_info.find("div",{"class": "clearFix"}).find_all("div",{"class": "viewRow"}):
    key = info.find("em")
    span = key.find("span")
    if span:
        span.extract()

     # Get the value of <p> tags
    value = info.find_all("p")  # Use find_all() to get a list of <p> tags
    
    key = " ".join(key.text.split())  # Clean up the key text
    
    # Extract and clean the text for each <p> tag inside value
    value_text = " ".join([v.text.strip() for v in value])  # Join the text from all <p> elements
    
    value_text = value_text.replace(" ,", ",")  # Replace " ," with ","
    
    meta_info[key] = value_text

In [104]:
print(key)
print(value_text)

진료과
소아청소년과
							
								,  
								산부인과


In [108]:
detail = medi_info.find("div",{"class":"detailWrap"})
keys = detail.find_all("h5")
values = detail.find_all("p")

detail_info ={}

for key, value in zip(keys,values):
    key = " ".join(key.text.split())
    value = " ".join(value.text.split())
    detail_info[key] = value

In [109]:
print(detail_info)

{'정의': '혈액형이 O형인 임신부가 A형 또는 B형인 아기를 가졌을 때 엄마가 가지고 있던 면역글로불린 G 유형(IgG type)의 항A, B항체(anti-A, B)가 태반을 건너가 아기의 적혈구 항원과 결합하여 비장에 있는 마크로파지(macrophage)에 의해 적혈구를 파괴되는 질환을 말한다.', '원인': '태아에게는 어머니에게는 없으나 아버지에게서 일방적으로 유전된 ABO 혈액형 항원이 있다. 아주 일부의 산모에게서 산전 혹은 산후에 태아-산모 출혈 등으로 인한 항체가 생성되며 이 항체(면역글로불린G, IgG)가 태반을 통과하여 신생아의 혈액 속에서 적혈구의 용혈을 일으킨다.', '증상': '가장 위험한 증상으로는 태아 수종(hydrops fetalis)이 있다. 용혈이 심해지면 빈혈 증상이 나타나고 이로 인해 저산소증이 발생하는데 이를 극복하기 위해 심박출량이 증가되어 심부전으로 빠질 수 있다. 결국 혈액량 증가, 정맥압 상승, 나트륨(Na) 및 염(Cl)의 체류, 전신 부종 등이 생긴다. 또한 파괴된 적혈구를 대체하기 위해 혈구 생성이 늘어나는데, 태아는 성인과 달리 골수 외에 다른 장기에서도 혈구 생성이 일어날 수 있다. 특히 간이나 비장에서 그러하며, 이런 장기들이 비대해지는 문제가 생긴다. 태아는 장기의 기능이 덜 성숙했기 때문에 과다한 일이 부여될 때 처리할 수 없는 상황이 발생할 수 있다. 적혈구가 파괴되면 빌리루빈이 생성되는데 빌리루빈은 간에서 포합과정(glucuronidation)을 거쳐 수용성으로 변하여 장관으로 배설되는 과정을 거친다. 빌리루빈이 너무 많아 제대로 포합과정을 거치지 않고 혈중에 많이 떠다니게 되고(과빌리루빈혈증), 포합과정을 거치지 않은 비포합 빌리루빈은 비수용성 물질로 뇌와 척수의 방어막인 혈뇌장벽(Blood-brain barrier)을 통과하여 핵황달(kernicterus)이 발생하게 된다. 핵황달이 심할 경우 태아는 사망할 수 있다.', '진단/검사': '신생아 혈액이나 제대 혈액에 대한 ABO, RhD 혈액

In [112]:
import json

with open("medic.json", 'w', encoding='utf-8') as f:
    json.dump(title, f, ensure_ascii=False, indent=2 )
    json.dump(meta_info, f, ensure_ascii=False, indent=2 )
    json.dump(detail_info, f, ensure_ascii=False, indent=2 )